In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import datasets, utils, callbacks
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers

import tensorflow.keras.backend as K

# 오토인코더

In [2]:
# 패션 mnist 데이터 셋을 통한 오토인코더 구현 실습
from tensorflow.keras import datasets
(X_train, y_train), (X_test, y_test) = datasets.fashion_mnist.load_data()

In [3]:
p = f'''
    X_train.shape = {X_train.shape}
    y_tarin.shape = {y_train.shape}
    
    X_test.shape = {X_test.shape}
    y_test.shape = {y_test.shape}
'''
print(p)


    X_train.shape = (60000, 28, 28)
    y_tarin.shape = (60000,)
    
    X_test.shape = (10000, 28, 28)
    y_test.shape = (10000,)



In [4]:
# 전처리
def preprocess(imgs):
    imgs = imgs.astype('float32') / 255.0 # rgb 값의 표준화
    imgs = np.pad(imgs, pad_width = 2, constant_values = 0.0) # pad_width의 두께로 패딩 추가
    imgs = np.expand_dims(imgs, axis = -1) # 차원을 확장하는 기능, axis는 어느 열에 차원을 추가할 것인지를 결정하는 것
    # axis = -1이면, 맨 마지막 열에 차원 추가
    
    return imgs

In [5]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

p = f'''
    X_train.shape = {X_train.shape}
    y_tarin.shape = {y_train.shape}
    
    X_test.shape = {X_test.shape}
    y_test.shape = {y_test.shape}
'''
print(p)


    X_train.shape = (60004, 32, 32, 1)
    y_tarin.shape = (60000,)
    
    X_test.shape = (10004, 32, 32, 1)
    y_test.shape = (10000,)



In [61]:
encoder_input = layers.Input(shape = (32, 32, 1), name = 'encoder_input')

x = layers.Conv2D(32, (3, 3), strides = 2, activation = 'relu', padding = 'same')(encoder_input)
x = layers.Conv2D(64, (3, 3), strides = 2, activation = 'relu', padding = 'same')(x)
x = layers.Conv2D(128, (3, 3), strides = 2, activation = 'relu', padding = 'same')(x)
shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x)

encoder_output = layers.Dense(2, name = 'encoder_output')(x) # output의 shape = (2, ) 따라서 디코더 input의 shape = (2,)

encoder = models.Model(encoder_input, encoder_output)

In [70]:
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
Total params: 96,770
Trainable params: 96,770
Non-trainable p

In [63]:
decoder_input = layers.Input(shape = (2,), name = 'decoder_input')

x = layers.Dense(np.prod(shape_before_flattening))(decoder_input) # np.prod() - 모든 원소들의 곱을 반환
x = layers.Reshape(shape_before_flattening)(x) # 인코더에서 플랫화하기 전의 데이터 형태로 변경
x = layers.Conv2DTranspose(128, (3, 3), strides = 2, activation = 'relu', padding = 'same')(x)
x = layers.Conv2DTranspose(64, (3, 3), strides = 2, activation = 'relu', padding = 'same')(x)
x = layers.Conv2DTranspose(32, (3, 3), strides = 2, activation = 'relu', padding = 'same')(x)

decoder_output = layers.Conv2D(1,
                               (3, 3),
                               strides = 1,
                               activation = 'sigmoid',
                               padding = 'same',
                               name = 'decoder_output')(x)

decoder = models.Model(decoder_input, decoder_output)

In [68]:
decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 2048)              6144      
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose_3 (Conv2DT  (None, 8, 8, 128)        147584    
 ranspose)                                                       
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 16, 16, 64)       73792     
 ranspose)                                                       
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 32, 32, 32)       1846

In [72]:
autoencoder = models.Model(encoder_input, decoder(encoder_output))

In [91]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
 model_1 (Functional)        (None, 32, 32, 1)         2462

In [92]:
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [93]:
autoencoder.fit(X_train,
                X_train,
                epochs = 3,
                batch_size = 100,
                shuffle = True,
                validation_data = (X_test, X_test)) # input 과 output이 같다!!!

Epoch 1/3
601/601 [==============================] - 60s 99ms/step - loss: 0.2878 - val_loss: 0.2617
Epoch 2/3
601/601 [==============================] - 55s 91ms/step - loss: 0.2572 - val_loss: 0.2616
Epoch 3/3
601/601 [==============================] - 57s 95ms/step - loss: 0.2544 - val_loss: 0.2563


In [94]:
autoencoder.save('./data/autoencoder')
encoder.save('./data/encoder')
decoder.save('./data/decoder')

INFO:tensorflow:Assets written to: ./data/autoencoder\assets


INFO:tensorflow:Assets written to: ./data/autoencoder\assets


INFO:tensorflow:Assets written to: ./data/encoder\assets


INFO:tensorflow:Assets written to: ./data/encoder\assets


INFO:tensorflow:Assets written to: ./data/decoder\assets


INFO:tensorflow:Assets written to: ./data/decoder\assets


In [6]:
autoencoder = models.load_model('./data/autoencoder')
encoder = models.load_model('./data/encoder')
decoder = models.load_model('./data/decoder')

In [7]:
# 이미지 재구성
example_images = X_test[:500]
predictions = autoencoder.predict(example_images)

16/16 [==============================] - 0s 9ms/step


### 잠재 공간 시각화하기

In [ ]:
embeddings = encoder.predict(example_images)

plt.figure(figsize = (8, 8))
plt.scatter(embeddings[:, 0], embeddings[:, 1], c = 'black', alpha = 0.5, s = 3)
plt.show()

16/16 [==============================] - 0s 2ms/step


In [ ]:
example_labels = y_test[:500]

figsize = 8
plt.figure(figsize = (figsize, figsize))
plt.scatter(embeddings[:, 0],
            embeddings[:, 1],
            cmap = 'rainbow',
            c = example_labels,
            alpha = 0.6,
            s = 3)
plt.colorbar()

plt.show()

### 새로운 이미지 생성하기

In [ ]:
mins, maxs = np.min(embeddings, axis = 0), np.max(embeddings, axis = 0)
sample = np.random.uniform(mins, maxs, size = (18, 2))
# 총 18 * 2의 숫자를 뽑는데, (x, y)의 형태로 18개를 뽑는 것

reconstructions = decoder.predict(sample)

In [9]:
np.random.uniform(0, 100, size = (10, 2))

array([[76.91425136,  1.56879972],
       [ 1.04864136, 33.18426395],
       [36.8614864 , 56.07546463],
       [82.12636056, 89.25478635],
       [43.49610641, 21.38876741],
       [94.13457316, 27.33449787],
       [ 7.22842278, 52.84174938],
       [44.01081147, 47.36034597],
       [ 7.95839605,  0.27944804],
       [58.98430633,  2.35548157]])